# Volatility Arbitrage Strategy

In [1]:
# import all necessary library
import pandas as pd 
import numpy as np 
import os

## Data Cleaning

- retrieve spx option data from Bloomberg
- retrieve spx index data from yhoo finance
- clean data
  - unnecessary rows and columns
  - option ticker breakdown
  - keep options with certain expiration date which have more than 4 strikes have been traded
  - calculate moneyness
  - save the term structure of options


In [27]:
# import option data csv file
os.chdir('D:/Github/volatility_arbitrage/SPX_option_price/20230901/')
option_data = pd.read_csv('spx_option_0901.csv')

option_data = option_data.dropna()

# ticker breakdown for call
option_data[['index', 'Expiry', 'strike']] = option_data['Ticker'].str.split(' ', expand=True)
option_data['Type'] = option_data['strike'].str[0]
option_data.drop(['index', 'strike'], axis=1, inplace=True)

# ticker breakdown for put
option_data[['index', 'Expiry.1', 'strike']] = option_data['Ticker.1'].str.split(' ', expand=True)
option_data['Type.1'] = option_data['strike'].str[0]
option_data.drop(['index', 'strike'], axis=1, inplace=True)

call_data = option_data.iloc[:, :7]
put_data = option_data.iloc[:, 7:]

print(option_data.head())
# print(call_data.head())
# print(put_data.head())


  Strike              Ticker         Bid         Ask        Last        IVM  \
1   4470  SPX 10/20/23 C4470  120.500000  122.100006  137.449997  12.142555   
2   4475  SPX 10/20/23 C4475  116.900009  118.500000  111.020004  12.066501   
3   4480  SPX 10/20/23 C4480  113.300003  114.900009    0.000000  11.984125   
4   4485  SPX 10/20/23 C4485  109.800003  111.300003  111.850006  11.903403   
5   4490  SPX 10/20/23 C4490  106.300003  107.900009    0.000000  11.832068   

    Volm  Strike.1            Ticker.1      Bid.1      Ask.1     Last.1  \
1    1.0    4470.0  SPX 10/20/23 P4470  50.100006  50.900009  50.400009   
2   18.0    4475.0  SPX 10/20/23 P4475  51.500000  52.300003  51.800003   
3    0.0    4480.0  SPX 10/20/23 P4480  52.799988  53.799988  54.400009   
4  600.0    4485.0  SPX 10/20/23 P4485  54.199997  55.199997  55.699997   
5    0.0    4490.0  SPX 10/20/23 P4490  55.699997  56.699997  60.680008   

       IVM.1  Volm.1    Expiry Type  Expiry.1 Type.1  
1  12.562401   102.

## Fix Implied Volatility Surface

### Find BS Implied Volatility

- use put-call parity to calculate the risk-free rate and dividend yield for spx index option
- use Black-Scholes model to calculate the implied volatility for each option

### Fit BS Implied Volatility Surface

- calculate the implied volatility for each option using the implied volatility of the at-the-money option with the same expiration date
- for the same expiration date, fit the implied volatility curve against spot moneyness using function below:
  - option 1: similar to tanh function
    $$
    \sigma(x)^2 = \sigma_{atm}^2 + \delta\displaystyle\left(\frac{\tanh(\kappa x)}{\kappa}\right) + \frac{\gamma}{2}\left(\frac{\tanh(\kappa x)}{\kappa}\right)^2,\quad
    \text{where }x = \displaystyle\ln(\frac{K}{S})
    $$
  - option 2: similar to sigmoid function
- for different expiration date, for same moneyness, connect the implied volatility curve using cubic spline interpolation

### Convert BS Implied Volatility Surface to Local Volatility Surface

- use Dupire's formula to calculate the local volatility for each option
![BS vs Local](./Public/BS_vs_local_var.png)

## Trading Strategy

- if the implied volatility of an option is higher than the local volatility, then sell the option

## Backtesting

# Reference

[SPX vs. SPXW Options: Understanding the Key Differences](https://insideoptions.io/spx-spxw-options-differences/)  
[Cubic Spline Interpolation](https://en.wikipedia.org/wiki/Spline_interpolation#Algorithm_to_find_the_interpolating_cubic_spline)